In [1]:
import pandas as pd
import numpy as np
import argparse
import os
import json
import time
import pickle
import random

import trinity.spec as S
from trinity.enumerator import LineSkeletonPriorityEnumerator
from trinity.enumerator import LineSkeletonEnumerator
from trinity.interpreter import MorpheusInterpreter, MorpheusCoarseAbstractInterpreter, MorpheusPartialInterpreter, MorpheusFineAbstractInterpreter
from trinity.decider import Example, MorpheusDecider
from trinity.synthesizer import MorpheusSynthesizer
from trinity.logger import get_logger
from trinity.recorder import get_recorder, set_recorder_dest
from trinity.profiler import get_profiler_pool

In [2]:
from test_morpheus_benchmark import load_example

logger = get_logger('trinity')
logger.setLevel('DEBUG')
set_recorder_dest(os.path.abspath("./logs"))
args = {
    "dsl": "test_min",
    "benchmark": 5,
    "skeletons": "test_min",
    "record": False,
}

In [3]:
# load dsl spec
spec = None
if args["dsl"].startswith("test"):
    spec = S.parse_file("./benchmarks/morpheus/{}/{}.dsl".format(args["benchmark"], args["dsl"]))
else:
    raise Exception("You should not reach here.")

# load benchmark
print("# loading benchmark...")
benchmark_example = load_example(args["benchmark"])

print("# input0 type: {}".format(benchmark_example.input[0].dtypes.tolist()))
print("# input0 is: {}".format(benchmark_example.input[0]))
print("# output type: {}".format(benchmark_example.output.dtypes.tolist()))
print("# output is: {}".format(benchmark_example.output))

[debug] Building Trinity spec from parse tree...
[debug] Processing type definitions...
[debug] Processing input/output definitions...
[debug] Processing function definitions...
[debug] Processing global predicates...


# loading benchmark...
# input0 type: [dtype('int64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64')]
# input0 is:    ID     T   P.1  P.2  Q.1
0   1  24.3  10.2  5.5  4.5
1   2  23.4  10.4  5.7  3.2
# output type: [dtype('int64'), dtype('O'), dtype('float64'), dtype('float64')]
# output is:    ID Channel     T     P
0   1       1  24.3  10.2
1   2       1  23.4  10.4
2   1       2  24.3   5.5
3   2       2  23.4   5.7
4   1       1  24.3   4.5
5   2       1  23.4   3.2


In [4]:
# cirtical: append scores to each production
s0 = 0
for p in spec.productions():
    # p.score = 1
    # p.score = random.random()
    p.score = s0
    s0 += 1

In [5]:
# load skeleton
print("# loading skeleton list...")
skeleton_list = None
if args["skeletons"].startswith("test"):
    with open("./benchmarks/morpheus/{}/{}_skeletons.json".format(args["benchmark"], args["skeletons"]), "r") as f:
        skeleton_list = json.load(f)
else:
    with open("./skeletons/{}_skeletons.json".format(args["skeletons"]), "r") as f:
        skeleton_list = json.load(f)

# loading skeleton list...


In [6]:
enumerator = LineSkeletonPriorityEnumerator( spec=spec, cands=skeleton_list )
# enumerator = LineSkeletonEnumerator( spec=spec, cands=skeleton_list )
interpreter = MorpheusInterpreter( spec=spec )
# coarse: perform skeleton level abstract interpretation, throws SkeletonAssertion
coarse_abstract_interpreter = MorpheusCoarseAbstractInterpreter()
# fine: perform finer level (sub-skeleton level) abstract interpretation throws EnumAssertion
fine_abstract_interpreter = MorpheusFineAbstractInterpreter( spec=spec )
partial_interpreter = MorpheusPartialInterpreter(
    interpreter=interpreter,
    # partial interpreter currently only works with coarse abstract interpreter
    # abstract_interpreter=fine_abstract_interpreter,
    abstract_interpreter=coarse_abstract_interpreter,
)

# print some basic info
# print("# search space (skeletons): {}".format(len(enumerator._iters)))
# print("# search space (concrete programs): {}".format(
#     sum([len(p) for p in enumerator._iters])
# ))
# input("press to start")
recorder = None

decider = MorpheusDecider( 
    interpreter=interpreter, 
    coarse_abstract_interpreter=coarse_abstract_interpreter,
    # fine_abstract_interpreter=fine_abstract_interpreter,
    partial_interpreter=partial_interpreter,
    examples=[benchmark_example], 
    equal_output=interpreter.equal_tb,
    recorder=recorder,
)
synthesizer = MorpheusSynthesizer(
    enumerator=enumerator,
    decider=decider
)

[info] The following types contain non-terminal production rules: Empty, Table. Try not to leave them as holes in your skeleton. Even though a spawning procedure can try to complete them, a cyclic substitution may still happen.


In [7]:
program = synthesizer.synthesize()

[debug] ---> select(separate(gather(@param0, ['0']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0']), 0), ['0']).
[debug] Rejected: EnumAssertion:partial:step1.
[debug] ---> select(separate(gather(@param0, ['1']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['1']), 0), ['0']).
[debug] Rejected: EnumAssertion:partial:step1.
[debug] ---> select(separate(gather(@param0, ['2']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['2']), 0), ['0']).
[debug] Rejected: EnumAssertion:partial:step1.
[debug] ---> select(separate(gather(@param0, ['3']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['3']), 0), ['0']).
[debug] Rejected: EnumAssertion:partial:step1.
[debug] ---> select(separate(gather(@param0, ['4']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['4']), 0), ['0']).
[debug] Rejected: EnumAssertion:partial:step1.
[debug] ---> select(separate(gather(@param0, ['5']), 0), ['0'])
[debug] Proposed: select(separa

# best score is: 152, from: [152]
# best score is: 153, from: [153]
# best score is: 154, from: [154]
# best score is: 155, from: [155]
# best score is: 156, from: [156]
# best score is: 157, from: [157]
# best score is: 158, from: [158]
# best score is: 159, from: [159]
# best score is: 160, from: [160]
# best score is: 161, from: [161]


[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['-4']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['-4']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['-5']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['-5']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '1']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '1']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '2']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '2']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '3']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '3']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0,

# best score is: 162, from: [162]
# best score is: 163, from: [163]
# best score is: 164, from: [164]
# best score is: 165, from: [165]
# best score is: 166, from: [166]
# best score is: 167, from: [167]
# best score is: 168, from: [168]
# best score is: 169, from: [169]


[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '-1']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '-1']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '-2']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '-2']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '-3']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '-3']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '-4']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '-4']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['0', '-5']), 0), ['0'])
[debug] Proposed: select(separate(gather(@param0, ['0', '-5']), 0), ['0']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> selec

...

# best score is: 293, from: [293]
# best score is: 294, from: [294]
# best score is: 295, from: [295]
# best score is: 296, from: [296]
# best score is: 297, from: [297]
# best score is: 298, from: [298]
# best score is: 299, from: [299]
# best score is: 300, from: [300]
# best score is: 301, from: [301]
# best score is: 302, from: [302]
# best score is: 303, from: [303]

[debug] ---> select(separate(gather(@param0, ['-4', '-5']), 5), ['-1', '-3'])
[debug] Proposed: select(separate(gather(@param0, ['-4', '-5']), 5), ['-1', '-3']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['-4', '-5']), 5), ['-1', '-4'])
[debug] Proposed: select(separate(gather(@param0, ['-4', '-5']), 5), ['-1', '-4']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['-4', '-5']), 5), ['-1', '-5'])
[debug] Proposed: select(separate(gather(@param0, ['-4', '-5']), 5), ['-1', '-5']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['-4', '-5']), 5), ['-2', '-3'])
[debug] Proposed: select(separate(gather(@param0, ['-4', '-5']), 5), ['-2', '-3']).
[debug] Rejected: EnumAssertion:gather:0.
[debug] ---> select(separate(gather(@param0, ['-4', '-5']), 5), ['-2', '-4'])
[debug] Proposed: select(separate(gather(@param0, ['-4', '-5']), 5), ['-2', '-4']).
[debug] Rejected: Enum


# best score is: 304, from: [304]
# best score is: 305, from: [305]
# best score is: 306, from: [306]
# best score is: 307, from: [307]
# best score is: 308, from: [308]
# best score is: 309, from: [309]
# best score is: 310, from: [310]
# best score is: 311, from: [311]


[info] Exhausted.


# best score is: None, from: [None]


In [9]:
enumerator._iters[0].get_next_best()

ApplyNode(select, [ApplyNode(separate, [ApplyNode(gather, [ParamNode(0), AtomNode(['1'])]), AtomNode(0)]), AtomNode(['0'])])

In [10]:
enumerator._iters[0].get_next_best()

ApplyNode(select, [ApplyNode(separate, [ApplyNode(gather, [ParamNode(0), AtomNode(['2'])]), AtomNode(0)]), AtomNode(['0'])])

In [116]:
enumerator._iters[0].get_next_best()

ApplyNode(select, [ApplyNode(separate, [ApplyNode(gather, [ParamNode(0), AtomNode(['-4', '-5'])]), AtomNode(5)]), AtomNode(['1', '5'])])

In [8]:
tmp = enumerator.next()[1]

In [9]:
tmp

ApplyNode(select, [ApplyNode(separate, [ApplyNode(gather, [ParamNode(0), AtomNode(['-99', '-3'])]), AtomNode(3)]), AtomNode(['0', '1'])])

In [12]:
tmp.is_apply()

True

In [11]:
tmp.children

[ApplyNode(separate, [ApplyNode(gather, [ParamNode(0), AtomNode(['-99', '-1'])]), AtomNode(4)]),
 AtomNode(['-99', '-1'])]

In [11]:
tmp.production.score

0.5325309219045944

In [13]:
enumerator._iters[0]

In [17]:
enumerator._iters[0]._piters[0].peek()

In [18]:
enumerator._iters[0]._piters[0]._ptr

78